# scrape web
zagor

# libraries
try not to add libraries globally, rather call f-ctions with ```library::f-ction```

In [1]:
library(magrittr)

# define the base URL

In [2]:
base_url = "https://projekt-spoznaj.si/videoposnetki-in-predstavitve/fair/"

# with rvest

## f-ctions

### find the total number of pages
*_Denote_*: page 1 has no number

In [3]:
find_total_pages = function(base_url) {
    current_page = 1
    has_next = TRUE
    last_page = 1
    
    while (has_next) {
    # Construct URL for the current page
    page_url = ifelse(current_page == 1, base_url, paste0(base_url, "page/", current_page, "/"))
    
    # Read the webpage
    webpage = tryCatch(rvest::read_html(page_url), error = function(e) return(NULL))
    
    if (is.null(webpage)) {
        break
    }
    
    # Check for the "Next" button
    next_button = webpage %>%
    rvest::html_nodes("a.next.page-numbers") %>%
    rvest::html_attr("href")
    
    if (length(next_button) == 0) {
    # No "Next" button found
    has_next = FALSE
    } else {
    # Increment the counter
    current_page = current_page + 1
    last_page = current_page
    }
    
}


return(last_page)
}


### scrape titles and dates

In [4]:
scrape_page = function(url) {
    
    # Read the webpage content
    webpage = rvest::read_html(url)
    
    # Extract lecture titles
    lecture_titles = webpage %>%
    rvest::html_nodes(".aiovg-title a.aiovg-link-title") %>%
    rvest::html_text()

    # Create a data.table for the extracted data
    data.table::data.table(
    Date = lecture_titles %>%
    stringr::str_extract("\\(\\d{1,2}\\. \\d{1,2}\\. \\d{4}\\)") %>% # Extract dates
    stringr::str_remove_all("[()]") %>% # Remove parentheses
    as.Date(format = "%d. %m. %Y"), # Convert to Date format
    Lecture_Title = lecture_titles %>%
    stringr::str_remove("\\s*\\(\\d{1,2}\\. \\d{1,2}\\. \\d{4}\\)") %>% # Remove dates from titles
    stringr::str_trim() # Remove whitespace and newlines
    )
}

## scrape

In [5]:
?rvest::html_nodes
?rvest::html_attr

total_pages = find_total_pages(base_url)
urls = c(base_url, paste0(base_url, "page/", 2:total_pages, "/"))
lectures_dt = data.table::rbindlist(lapply(urls, scrape_page))

rename                  package:rvest                  R Documentation

_F_u_n_c_t_i_o_n_s _r_e_n_a_m_e_d _i_n _r_v_e_s_t _1._0._0

_D_e_s_c_r_i_p_t_i_o_n:

     *[Deprecated]*

     rvest 1.0.0 renamed a number of functions to ensure that every
     function has a common prefix, matching tidyverse conventions that
     emerged since rvest was first created.

        * 'set_values()' -> 'html_form_set()'

        * 'submit_form()' -> 'session_submit()'

        * 'xml_tag()' -> 'html_name()'

        * 'xml_node()' & 'html_node()' -> 'html_element()'

        * 'xml_nodes()' & 'html_nodes()' -> 'html_elements()'

     ('html_node()' and 'html_nodes()' are only superseded because
     they're so widely used.)

     Additionally all session related functions gained a common prefix:

        * 'html_session()' -> 'session()'

        * 'forward()' -> 'session_forward()'

        * 'back()' -> 'session_back()'

        * 'jump_to()' -> 'session_jump_to(

html_attr                package:rvest                 R Documentation

_G_e_t _e_l_e_m_e_n_t _a_t_t_r_i_b_u_t_e_s

_D_e_s_c_r_i_p_t_i_o_n:

     'html_attr()' gets a single attribute; 'html_attrs()' gets all
     attributes.

_U_s_a_g_e:

     html_attr(x, name, default = NA_character_)
     
     html_attrs(x)
     
_A_r_g_u_m_e_n_t_s:

       x: A document (from 'read_html()'), node set (from
          'html_elements()'), node (from 'html_element()'), or session
          (from 'session()').

    name: Name of attribute to retrieve.

 default: A string used as a default value when the attribute does not
          exist in every element.

_V_a_l_u_e:

     A character vector (for 'html_attr()') or list ('html_attrs()')
     the same length as 'x'.

_E_x_a_m_p_l_e_s:

     html <- minimal_html('<ul>
       <li><a href="https://a.com" class="important">a</a></li>
       <li class="active"><a href="https://c.com">b</a></li>
     

##  write res

In [6]:
fp = file.path('..', 'output')
fn = "R-lecture-titles-dates.xlsx"
openxlsx::write.xlsx(lectures_dt, file.path(fp, fn))

# with xml2

### find the total number of pages

In [7]:
find_total_pages = function(base_url) {

    current_page = 1
    has_next = TRUE
    last_page = 1

    while (has_next) {
    # URL
    page_url = ifelse(current_page == 1, base_url, paste0(base_url, "page/", current_page, "/"))
    
    # Fetch content
    response = httr::GET(page_url)
    
    # successful?
    if (httr::status_code(response) != 200) {
    break
    }
    
    # Parse
    webpage = xml2::read_html(httr::content(response, as = "text"))
    
    # "Next" button?
    next_button = xml2::xml_find_first(webpage, "//a[contains(@class, 'next page-numbers')]")
    if (is.na(next_button)) {
    # No "Next" button
    has_next = FALSE
    } else {
    # counter
    current_page = current_page + 1
    last_page = current_page
    }
}

return(last_page)
}

### scripe titles and dates

In [8]:
scrape_page <- function(url) {
    # webpage content
    response = httr::GET(url)

    # successful ?
    if (httr::status_code(response) != 200) {
    return(data.table::data.table(Date = as.Date(character()), Lecture_Title = character()))
    }

    # Parse
    webpage <- xml2::read_html(httr::content(response, as = "text"))

    # titles
    titles_nodes <- xml2::xml_find_all(webpage, "//div[contains(@class, 'aiovg-title')]//a[contains(@class, 'aiovg-link-title')]")
    lecture_titles <- xml2::xml_text(titles_nodes)

    # dates
    dates <- stringr::str_extract(lecture_titles, "\\(\\d{1,2}\\. \\d{1,2}\\. \\d{4}\\)") %>%
    stringr::str_remove_all("[()]") %>%
    as.Date(format = "%d. %m. %Y")

    # clean titles
    cleaned_titles <- stringr::str_remove(lecture_titles, "\\s*\\(\\d{1,2}\\. \\d{1,2}\\. \\d{4}\\)") %>%
    stringr::str_trim()

    # data table
    return(data.table::data.table(Date = dates, Lecture_Title = cleaned_titles))
}

### ml structure

In [9]:
?xml2::xml_find_all

(response = httr::GET(base_url))
webpage = xml2::read_html(httr::content(response, as = "text"))
webpage


Response [https://projekt-spoznaj.si/videoposnetki-in-predstavitve/fair/]
  Date: 2024-11-18 08:36
  Status: 200
  Content-Type: text/html; charset=UTF-8
  Size: 169 kB
<!DOCTYPE html>
<!--[if IE 7]><html class="ie ie7" lang="sl-SI">
<![endif]-->
<!--[if IE 8]><html class="ie ie8" lang="sl-SI">
<![endif]-->
<!--[if !(IE 7) | !(IE 8)  ]><!--><html lang="sl-SI" class="no-js">
<!--<![endif]--><head><script data-no-optimize="1">var litespeed_docref=sessi...
body {
font-family : 'Roboto'!important;
}</style><style id="wfc-style-fonts-menu_items" type="text/css" data-origin="...
...

{html_document}
<html lang="sl-SI" class="no-js">
[1] <head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8 ...
[2] <body class="nb-3-3-5 nb-pro-1-0-17 nimble-has-local-data-skp__post_page_ ...

xml_find_all               package:xml2                R Documentation

_F_i_n_d _n_o_d_e_s _t_h_a_t _m_a_t_c_h _a_n _x_p_a_t_h _e_x_p_r_e_s_s_i_o_n.

_D_e_s_c_r_i_p_t_i_o_n:

     Xpath is like regular expressions for trees - it's worth learning
     if you're trying to extract nodes from arbitrary locations in a
     document. Use 'xml_find_all' to find all matches - if there's no
     match you'll get an empty result. Use 'xml_find_first' to find a
     specific match - if there's no match you'll get an 'xml_missing'
     node.

_U_s_a_g_e:

     xml_find_all(x, xpath, ns = xml_ns(x), ...)
     
     ## S3 method for class 'xml_nodeset'
     xml_find_all(x, xpath, ns = xml_ns(x), flatten = TRUE, ...)
     
     xml_find_first(x, xpath, ns = xml_ns(x))
     
     xml_find_num(x, xpath, ns = xml_ns(x))
     
     xml_find_int(x, xpath, ns = xml_ns(x))
     
     xml_find_chr(x, xpath, ns = xml_ns(x))
     
     xml_find_lgl(x, xpath, 

### scrape

In [10]:
total_pages = find_total_pages(base_url)
head(total_pages)

urls = c(base_url, paste0(base_url, "page/", 2:total_pages, "/"))
head(urls)

lectures_dt = data.table::rbindlist(lapply(urls, scrape_page))
head(lectures_dt)

[1] 2

[1] "https://projekt-spoznaj.si/videoposnetki-in-predstavitve/fair/"       
[2] "https://projekt-spoznaj.si/videoposnetki-in-predstavitve/fair/page/2/"

Date,Lecture_Title
<date>,<chr>
2023-06-23,Odprta znanost v Evropskem raziskovalnem prostoru
2023-09-07,"Odprta znanost v Evropskem raziskovalnem prostoru, ponovitev"
2023-09-14,Nacionalna zakonodaja na področju odprte znanosti
2023-09-21,Slovenska skupnost odprte znanosti
2023-09-28,Možnosti in ugodnosti pri odprtem objavljanju za slovensko akademsko skupnost
2023-10-12,Platforma za odprte objave Open Research Europe


# session info

In [11]:
sessionInfo()

R version 4.3.1 (2023-06-16 ucrt)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 10 x64 (build 19045)

Matrix products: default


locale:
[1] LC_COLLATE=English_United Kingdom.utf8 
[2] LC_CTYPE=English_United Kingdom.utf8   
[3] LC_MONETARY=English_United Kingdom.utf8
[4] LC_NUMERIC=C                           
[5] LC_TIME=English_United Kingdom.utf8    

time zone: Europe/Ljubljana
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] magrittr_2.0.3

loaded via a namespace (and not attached):
 [1] zip_2.3.1         crayon_1.5.3      vctrs_0.6.5       httr_1.4.7       
 [5] cli_3.6.3         rlang_1.1.4       stringi_1.8.4     data.table_1.16.0
 [9] jsonlite_1.8.8    glue_1.7.0        selectr_0.4-2     htmltools_0.5.8.1
[13] IRdisplay_1.1     IRkernel_1.3.2    fansi_1.0.6       evaluate_0.24.0  
[17] fastmap_1.2.0     base64enc_0.1-3   openxlsx_4.2.7    lifecycle_1.0.4 